In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 700)

In [3]:
us_state_abbrev = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS',
                   'KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC',
                   'ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']



In [4]:
# # Empty DataFrame to add to in the loop below
# df = pd.DataFrame()

# for i in us_state_abbrev:
#     df_i = pd.read_html('https://www.walkscore.com/' + i)[0]
#     df_i['State'] = i
#     df = pd.concat([df, df_i])

In [6]:
df = pd.read_csv('walk_score.csv')
print(df.shape)
df.head()

(2500, 8)


,Unnamed: 0,City,Zip Code,Walk Score,Transit Score,Bike Score,Population,State
0,0,Birmingham (the largest city in Alabama),35211.0,35,25,31,212237,AL
1,1,Montgomery,36109.0,26,16,38,205764,AL
2,2,Mobile,36605.0,32,--,39,195111,AL
3,3,Huntsville,35810.0,24,13,40,180105,AL
4,4,Tuscaloosa,NaN,33,--,37,90468,AL


In [7]:
# df.to_csv('walk_score.csv')

In [8]:
df.isnull().sum()

Unnamed: 0          0
City                0
Zip Code         1489
Walk Score          0
Transit Score       0
Bike Score          0
Population          0
State               0
dtype: int64

In [9]:
df[['Walk Score', 'Transit Score', 'Bike Score']] = df[['Walk Score', 'Transit Score', 'Bike Score']].replace('--', np.NaN)

In [10]:
df.isnull().sum()

Unnamed: 0          0
City                0
Zip Code         1489
Walk Score          0
Transit Score    1442
Bike Score          0
Population          0
State               0
dtype: int64

In [11]:
df[['Walk Score', 'Transit Score', 'Bike Score']] = df[['Walk Score', 'Transit Score', 'Bike Score']].astype(int)

ValueError: cannot convert float NaN to integer